In [ ]:
# The packages needed for this project of prediting flight delays
#from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
#import pyspark.sql as sparksql
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pyspark.sql.functions import isnan
from pyspark.sql.functions import col, sum
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()
import sys
pd.options.display.float_format = '{:.0f}'.format

In [ ]:
# This is read the data from csv and create a data frame
df = pd.read_csv('DelayedFlight.csv')

In [ ]:
df

In [ ]:
#first five rows of the dataset
df.head()

In [ ]:
#last 5 rows of the dataset
df.tail()

In [ ]:
#give some information on columns types and number of null values
print('Dataframe dimensions:', df.shape)
#tab_info=pd.DataFrame(df.dtypes).T.rename(index={0:'column type'})
#tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
#tab_info=tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100)
                         .T.rename(index={0:'null values (%)'}))
#tab_info

In [ ]:
#This shows the correlations of the variables target Variables is ÄrrDelay""
df.corr()


In [ ]:
print (df.info()) #data set info

In [ ]:
#get the total number of missing values in dataframe
print(df.isnull().sum())

In [ ]:
df.shape

In [ ]:
#causes of delay
grouped = df[['ArrDelay','CarrierDelay', 
        'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']].groupby('ArrDelay').mean()
grouped.plot(kind='hist')

In [ ]:
#causes of delay
grouped = df[['ArrDelay','UniqueCarrier']].groupby('UniqueCarrier').count()
grouped.plot(kind='hist')

In [ ]:
#causes of delay by month
grouped = df[['ArrDelay','Month']].groupby('Month').count()
grouped.plot(kind='bar')

In [ ]:
# Compute average number of delayed flights by hour
df['hour'] = df['CRSDepTime'].map(lambda x: int(str(int(x)).zfill(4)[:2]))
grouped = df[['ArrDelay', 'hour']].groupby('hour').mean()

# plot average delays by hour of day
grouped.plot(kind='bar')

In [ ]:
# Compute average number of delayed flights per carrier
grouped1 = df[['ArrDelay', 'UniqueCarrier']].groupby('UniqueCarrier').filter(lambda x: len(x)>10)
grouped2 = grouped1.groupby('UniqueCarrier').mean()
UniqueCarrier = grouped2.sort_values(['ArrDelay'], ascending=False)

# display top 15 destination carriers by delay (from ORD)
UniqueCarrier[:15].plot(kind='bar')

In [ ]:
# Compute average number of delayed flights per carrier
grouped1 = df[['ArrDelay', 'Dest']].groupby('Dest').filter(lambda x: len(x)>10)
grouped2 = grouped1.groupby('Dest').mean()
Dest = grouped2.sort_values(['ArrDelay'], ascending=False)

# display top 15 destination carriers by delay (from ORD)
Dest[:15].plot(kind='bar')

In [ ]:
print ("ratio = ", str( (70882*1.0) / (362416+70882) )) #show the ratio

In [ ]:
#unique carrier vs dep delay time - shows which carrier got highest departure time
sns.set_style('whitegrid')
carrier_order = np.unique(list(df['UniqueCarrier']))
sns.barplot(x='UniqueCarrier',y ='DepDelay', data=df, color = "green" , order = carrier_order)
fig = plt.title("Carrier Vs Dep Delay")

In [ ]:
for i in df.columns:
    print (i, df[i].nunique())
    #show the type of interger data fields

In [ ]:
print ("ratio = ", str( (70882*1.0) / (362416+70882) )) #show the ratio

In [ ]:
#unique carrier vs dep delay time - shows which carrier got highest departure time
sns.set_style('whitegrid')
carrier_order = np.unique(list(df['UniqueCarrier']))
sns.barplot(x='UniqueCarrier',y ='DepDelay', data=df, color = "green" , order = carrier_order)
fig = plt.title("Carrier Vs Dep Delay")

In [ ]:
# dropping columns which do not hold any useful data and columns that contain repeated information
df_clean = df.drop = ['ID', 'Year', 'Month', 'DayOfMonth' 'DayOfWeek',
                      'DepTime', 'ArrTime' 'CRSDepTime', 'CRSArrTime', 'UniqueCarrier',
                       'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 
                     'Origin', 'Cancelled','CancellationCode']
#df.drop(variables_to_remove, axis = 1)
#df = df[['ArrDelay','DepDelay', 'CarrierDelay','LateAircraftDelay',
 #       'NASDelay','TaxiOut', 'WeatherDelay', 'TaxiIn', 'FlightNum', 
  #       'AirTime', 'Distance', 'SecurityDelay','Dest']]


In [ ]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

In [ ]:
#get the total number of missing values in dataframe
df.shape